<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#question-1" data-toc-modified-id="question-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>question 1</a></span></li><li><span><a href="#question-2" data-toc-modified-id="question-2-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>question 2</a></span></li></ul></div>

In [85]:
import numpy as np
import re
from copy import deepcopy

In [53]:
rawdata = """\
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
""".split("\n\n")
rawflows, rawinput = rawdata
rawflows, rawinput

('px{a<2006:qkq,m>2090:A,rfg}\npv{a>1716:R,A}\nlnx{m>1548:A,A}\nrfg{s<537:gd,x>2440:R,A}\nqs{s>3448:A,lnx}\nqkq{x<1416:A,crn}\ncrn{x>2662:A,R}\nin{s<1351:px,qqz}\nqqz{s>2770:qs,m<1801:hdj,R}\ngd{a>3333:R,R}\nhdj{m>838:A,pv}',
 '{x=787,m=2655,a=1222,s=2876}\n{x=1679,m=44,a=2067,s=496}\n{x=2036,m=264,a=79,s=2244}\n{x=2461,m=1339,a=466,s=291}\n{x=2127,m=1623,a=2188,s=1013}\n')

In [ ]:
with open("input.txt") as f:
    rawflows, rawinput = f.read().split("\n\n")

In [55]:
rawflows.split("\n")

['px{a<2006:qkq,m>2090:A,rfg}',
 'pv{a>1716:R,A}',
 'lnx{m>1548:A,A}',
 'rfg{s<537:gd,x>2440:R,A}',
 'qs{s>3448:A,lnx}',
 'qkq{x<1416:A,crn}',
 'crn{x>2662:A,R}',
 'in{s<1351:px,qqz}',
 'qqz{s>2770:qs,m<1801:hdj,R}',
 'gd{a>3333:R,R}',
 'hdj{m>838:A,pv}']

In [56]:
def parse_flow(rawflows):
    flows = dict()
    for line in rawflows.split("\n"):
        key, flowraw = line.rstrip("}").split("{")
        #print(key, flowraw)
        steps = []
        pattern = r'(\w+|[<>])'
        for step in flowraw.split(","):
            if ":" in step:
                condraw, name = step.split(":")
                cond = re.findall(pattern, condraw)
                cond[2] = int(cond[2])
                steps.append([name, cond])
            else:
                steps.append([step])
        flows[key] = steps
    return flows

In [57]:
flows = parse_flow(rawflows)
flows['in']

[['px', ['s', '<', 1351]], ['qqz']]

In [58]:
def parse_input(inp):
    V = inp.strip("{}").split(",")
    return [ int(item.split("=")[-1]) for item in V ]
parts = [ parse_input(inp) for inp in rawinput[:-1].split("\n") ]
parts[-3:]

[[2036, 264, 79, 2244], [2461, 1339, 466, 291], [2127, 1623, 2188, 1013]]

## question 1

In [8]:
def get_new_flow(part, flow):
    I = {'x':0, 'm':1, 'a':2, 's':3}
    for key, condition in flow[:-1]:
        cat, op, v = condition # category, operation, value
        vp = part[I[cat]]
        #print(cat, op, v, vp)
        if (op=="<" and vp<v) or (op==">" and vp>v):
            new_flow = key
            return new_flow
    else:
        return flow[-1][0]

def get_state(part):
    #print(part)
    flow = flows['in']
    maxit, nit = 10, 0
    while True:
        new_flow_key = get_new_flow(part, flow)
        #print(part, new_flow_key)
        if new_flow_key == "A":
            return "A"
        elif new_flow_key == "R":
            return "R"
        else:
            flow = flows[new_flow_key]
        
        nit += 1
        if nit >= maxit:
            print("reached maxit")
            break
        print(nit, end=",")

In [ ]:
total = 0
for part in parts:
    if get_state(part) == "A":
        total += sum(part)
total

## question 2

In [59]:
class Range(object):
    def __init__(self, start=1, end=4000):
        self.start = start
        self.end = end
    def split(self, op, cv): #operator condition_value
        # return in order of succes / no success
        # this has to split more carefully. also return [None, Range] or [Range, None] to indicate what the next step will be.
        if op=="<":
            if self.start>=cv:
                return [None, self]
            elif self.end<cv:
                return [self, None]
            else:
                return Range(start=self.start, end=cv-1), Range(start=cv, end=self.end)
        else:
            assert op==">"
            if self.start>cv:
                return [self, None]
            if self.end<=cv:
                return [None, self]
            else:
                return Range(start=cv+1, end=self.end), Range(start=self.start, end=cv)
    def __repr__(self):
        return f"<R:{self.start}-{self.end}>"
    def __len__(self):
        return self.end - self.start + 1

In [104]:
def handle_ranges(ranges, flow):
    print(ranges, flow)
    for key, condition in flow[:-1]:
        cat, op, v = condition # category, operation, value
        ranges_success, ranges_fail = ranges[I[cat]].split(op, v)
        print("in for loop", key, ":", cat, op, v, ranges_success, ranges_fail)
        if ranges_success: # so apply the condition, i.e. handle key
            new_ranges = deepcopy(ranges)
            new_ranges[I[cat]] = ranges_success
            if key == "A":
                accepted_ranges.append(new_ranges)
                return
            elif key== "R":
                return
            else:
                handle_ranges(new_ranges, flows[key])
        if ranges_fail: # the ranges with the fail part goes to next iteration
            ranges[I[cat]] = ranges_fail
            print("ranges after fail", ranges)
    else: # last flow
        new_flow = flow[-1][0]
        print("last flow with ranges", ranges, "and new_flow", new_flow)
        if new_flow == "A":
            accepted_ranges.append(ranges)
            return
        elif new_flow == "R":
            return
        else:
            return handle_ranges(ranges.copy(), flows[new_flow])

In [105]:
initial = [Range(), Range(), Range(), Range()]
accepted_ranges = []
I = {'x':0, 'm':1, 'a':2, 's':3}
handle_ranges(initial, flows['in'])

[<R:1-4000>, <R:1-4000>, <R:1-4000>, <R:1-4000>] [['px', ['s', '<', 1351]], ['qqz']]
in for loop px : s < 1351 <R:1-1350> <R:1351-4000>
[<R:1-4000>, <R:1-4000>, <R:1-4000>, <R:1-1350>] [['qkq', ['a', '<', 2006]], ['A', ['m', '>', 2090]], ['rfg']]
in for loop qkq : a < 2006 <R:1-2005> <R:2006-4000>
[<R:1-4000>, <R:1-4000>, <R:1-2005>, <R:1-1350>] [['A', ['x', '<', 1416]], ['crn']]
in for loop A : x < 1416 <R:1-1415> <R:1416-4000>
ranges after fail [<R:1-4000>, <R:1-4000>, <R:2006-4000>, <R:1-1350>]
in for loop A : m > 2090 <R:2091-4000> <R:1-2090>
ranges after fail [<R:1-4000>, <R:1-4000>, <R:1-4000>, <R:1351-4000>]
last flow with ranges [<R:1-4000>, <R:1-4000>, <R:1-4000>, <R:1351-4000>] and new_flow qqz
[<R:1-4000>, <R:1-4000>, <R:1-4000>, <R:1351-4000>] [['qs', ['s', '>', 2770]], ['hdj', ['m', '<', 1801]], ['R']]
in for loop qs : s > 2770 <R:2771-4000> <R:1351-2770>
[<R:1-4000>, <R:1-4000>, <R:1-4000>, <R:2771-4000>] [['A', ['s', '>', 3448]], ['lnx']]
in for loop A : s > 3448 <R:3449

In [106]:
flows['in'], flows['px'], flows['qkq'], flows['qqz'], flows['qs'], flows['pv']

([['px', ['s', '<', 1351]], ['qqz']],
 [['qkq', ['a', '<', 2006]], ['A', ['m', '>', 2090]], ['rfg']],
 [['A', ['x', '<', 1416]], ['crn']],
 [['qs', ['s', '>', 2770]], ['hdj', ['m', '<', 1801]], ['R']],
 [['A', ['s', '>', 3448]], ['lnx']],
 [['R', ['a', '>', 1716]], ['A']])

In [109]:
accepted_ranges

[[<R:1-1415>, <R:1-4000>, <R:1-2005>, <R:1-1350>],
 [<R:1-4000>, <R:2091-4000>, <R:2006-4000>, <R:1-1350>],
 [<R:1-4000>, <R:1-4000>, <R:1-4000>, <R:3449-4000>],
 [<R:1-4000>, <R:839-1800>, <R:1-4000>, <R:1351-2770>]]

In [110]:
ntotal = 0
for accepted_range in accepted_ranges:
    lens = list(map(len, accepted_range))
    print(lens, np.prod(lens))
    ntotal += np.prod(lens)
ntotal

[1415, 4000, 2005, 1350] 15320205000000
[4000, 1910, 1995, 1350] 20576430000000
[4000, 4000, 4000, 552] 35328000000000
[4000, 962, 4000, 1420] 21856640000000


93081275000000

In [92]:
# correct answer should be 
# 167409079868000
# my answers are:
# 227579675642736
# 212938445310001
# 145370496664781
#  93081275000000
# 150265915000000